In [ ]:
--!jinja
ENV = '{{ env }}'
print("Running notebook in environment:", ENV)

# Optionally, set the session variable so subsequent SQL statements in the notebook can use it.
from snowflake.snowpark import Session
session = Session.get_active_session()
session.sql(f"ALTER SESSION SET ENV = '{ENV}'").collect()

DB_NAME = "FRED_DB"

In [ ]:
# Use RAW_DAILY for both staging and tables

TARGET_SCHEMA = f"{ENV}_RAW_SCHEMA"
 
# S3 paths for the CSV files

STAGES = {

    "DAILYCURRENCY_RAW_STAGE": "s3://fredcurrencyexhange/DailyCurrencyExchange/",

    "MONTHLYCURRENCYEXCHANGE_RAW_STAGE": "s3://fredcurrencyexhange/MonthlyCurrencyExchange/"

}
 
# Create the target schema if it doesn't exist

session.sql(f"CREATE SCHEMA IF NOT EXISTS {DB_NAME}.{TARGET_SCHEMA}").collect()

print(f"✅ Schema created/verified for {TARGET_SCHEMA} environment")
 
# Create CSV file format in the target schema

create_file_format_sql = f"""

CREATE OR REPLACE FILE FORMAT {TARGET_SCHEMA}.CSV_FORMAT

    TYPE = 'CSV'

    FIELD_DELIMITER = ','

    PARSE_HEADER = TRUE

    SKIP_BLANK_LINES = TRUE

    TRIM_SPACE = TRUE

    ENCODING = 'UTF-8'

    NULL_IF = ('.', 'NULL', 'null', '');

"""

session.sql(create_file_format_sql).collect()

print("✅ Created/Verified CSV file format in", TARGET_SCHEMA)
 
# Create external stages in the target schema (directly loading from S3)

for stage, s3_path in STAGES.items():

    create_stage_sql = f"""

    CREATE OR REPLACE STAGE {TARGET_SCHEMA}.{stage}

    STORAGE_INTEGRATION = fred_s3_integration

    URL = '{s3_path}'

    FILE_FORMAT = {TARGET_SCHEMA}.CSV_FORMAT;

    """

    session.sql(create_stage_sql).collect()

    print(f"✅ Created/Verified stage: {stage} in {TARGET_SCHEMA}")
 
print("🔄 Snowflake stages setup completed.")
 
# List files in the stages and load them into tables (all in TARGET_SCHEMA)

for stage in STAGES:

    print(f"\n📁 Files in {stage}:")

    list_files_sql = f"LIST @{TARGET_SCHEMA}.{stage};"

    files = session.sql(list_files_sql).collect()

    if not files:

        print("  No files found in stage.")

    else:

        for file in files:

            file_path = file[0]  # Full file path in S3

            file_size = file[1]  # File size in bytes

            file_name = os.path.basename(file_path)  # Extract just the file name

            print(f"  {file_path} (Size: {file_size} bytes)")

            # Generate table name (upper-case, without extension)

            table_name = file_name.split(".")[0].upper()

            # Create table in the TARGET_SCHEMA

            create_table_sql = f"""

            CREATE OR REPLACE TABLE {DB_NAME}.{TARGET_SCHEMA}.{table_name} (

                DATE DATE,

                VALUE FLOAT

            );

            """

            session.sql(create_table_sql).collect()

            print(f"✅ Created table: {table_name} in {TARGET_SCHEMA}")
 
            # Load data from the S3 stage into the table

            copy_into_sql = f"""

            COPY INTO {DB_NAME}.{TARGET_SCHEMA}.{table_name} 

            FROM @{TARGET_SCHEMA}.{stage}/{file_name}

            FILE_FORMAT = {TARGET_SCHEMA}.CSV_FORMAT

            MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE;

            """

            session.sql(copy_into_sql).collect()

            print(f"✅ Data loaded into {TARGET_SCHEMA}.{table_name}")
 
            # Update table to replace NULLs (from '.') with 0

            update_sql = f"""

            UPDATE {DB_NAME}.{TARGET_SCHEMA}.{table_name}

            SET VALUE = 0

            WHERE VALUE IS NULL;

            """

            session.sql(update_sql).collect()

            print(f"✅ Updated {table_name}: replaced NULL with 0 in VALUE column")

 